# How AES Works

## Keyed Permutations

AES, como todos los buenos cifradores de bloques, realiza una "permutación con clave", es decir, asigna cada bloque de entrada posible a un bloque de salida único, con una clave que determina qué permutación se debe realizar.

> Un "bloque" se refiere simplemente a una cantidad fija de bits o bytes, que pueden representar cualquier tipo de datos. AES procesa un bloque y genera otro bloque. Hablaremos específicamente de la variante de AES que funciona con bloques de 128 bits (16 bytes) y una clave de 128 bits, conocida como AES-128.

Usando la misma clave, la permutación puede realizarse a la inversa, mapeando el bloque de salida nuevamente al bloque de entrada original. Es importante que exista una correspondencia uno a uno entre los bloques de entrada y salida, de otra manera no podríamos confiar en el texto cifrado para descifrarlo nuevamente al mismo texto simple con el que comenzamos.

¿Cuál es el término matemático para una correspondencia uno a uno?

In [ ]:
# Una biyección (bijection) es una correspondencia entre dos conjuntos que asigna a cada elemento de un conjunto un único elemento del otro conjunto

## Resisting Bruteforce

Si un cifrado de bloques es seguro, no debería haber forma de que un atacante distinga la salida de AES de una permutación aleatoria de bits. Además, no debería haber una mejor forma de deshacer la permutación que simplemente forzando bruscamente cada clave posible. Es por eso que los académicos consideran que un cifrado está teóricamente "roto" si pueden encontrar un ataque que requiera menos pasos para realizarse que forzar bruscamente la clave, incluso si ese ataque es prácticamente inviable.

> ¿Qué tan difícil es forzar un espacio de claves de 128 bits? [Alguien calculó](https://crypto.stackexchange.com/questions/48667/how-long-would-it-take-to-brute-force-an-aes-128-key/48669#48669) que si se utilizara la potencia de toda la red de minería de Bitcoin contra una clave AES-128, se necesitarían cien veces la edad del universo para descifrar la clave.

Resulta que existe un [ataque](https://en.wikipedia.org/wiki/Biclique_attack) contra AES que es mejor que el de fuerza bruta, pero solo un poco: reduce el nivel de seguridad de AES-128 a 126,1 bits y no se ha mejorado en más de 8 años. Dado el gran "margen de seguridad" que proporcionan los 128 bits y la falta de mejoras a pesar de un estudio exhaustivo, no se considera un riesgo creíble para la seguridad de AES. Pero sí, en un sentido muy estricto, "rompe" AES.

Finalmente, si bien las computadoras cuánticas tienen el potencial de romper por completo los criptosistemas de clave pública populares como RSA a través del [algoritmo de Shor](https://es.wikipedia.org/wiki/Algoritmo_de_Shor), se cree que solo reducen a la mitad el nivel de seguridad de los criptosistemas simétricos a través del [algoritmo de Grover](https://es.wikipedia.org/wiki/Algoritmo_de_Grover). Esta es una de las razones por las que la gente recomienda usar AES-256, a pesar de que tiene un rendimiento menor, ya que aún proporcionaría una seguridad de 128 bits muy adecuada en un futuro cuántico.

¿Cuál es el nombre del mejor ataque de clave única contra AES?

In [ ]:
# El ataque biclique es una variante del método de criptoanálisis de encuentro en el medio (MITM).
# Utiliza una estructura biclique para extender el número de rondas que pueden ser atacadas por el ataque MITM.

## Structure of AES

Para lograr una permutación con clave que no sea factible de invertir sin la clave, AES aplica una gran cantidad de operaciones de mezcla ad hoc en la entrada. Esto contrasta marcadamente con los criptosistemas de clave pública como RSA, que se basan en elegantes problemas matemáticos individuales. AES es mucho menos elegante, pero es muy rápido.

En un nivel alto, AES-128 comienza con un "programa de clave" y luego ejecuta 10 rondas sobre un estado. El estado inicial es simplemente el bloque de texto simple que queremos cifrar, representado como una matriz de bytes de 4x4. A lo largo de las 10 rondas, el estado se modifica repetidamente mediante una serie de transformaciones invertibles.

> Cada paso de transformación tiene un propósito definido basado en las propiedades teóricas de los cifrados seguros establecidos por Claude Shannon en la década de 1940. Analizaremos cada uno de ellos más de cerca en los siguientes desafíos.

A continuación se muestra una descripción general de las fases del cifrado AES:

1. **KeyExpansion o Key Schedule**

A partir de la clave de 128 bits, se derivan 11 "claves de ronda" independientes de 128 bits: una para utilizar en cada paso AddRoundKey.

2. **Adición de clave inicial**

AddRoundKey: los bytes de la clave de la primera ronda se combinan con los bytes del estado.

3. **Ronda**: esta fase se repite 10 veces, para 9 rondas principales más una "ronda final"

  a) SubBytes: cada byte del estado se sustituye por un byte diferente según una tabla de búsqueda ("S-box").

  b) ShiftRows: las últimas tres filas de la matriz de estado se transponen (se desplazan una, dos o tres columnas).

  c) MixColumns: se realiza la multiplicación de matrices en las columnas del estado, combinando los cuatro bytes de cada columna. Esto se omite en la ronda final.

  d) AddRoundKey: los bytes de la clave de la ronda actual se combinan con los bytes del estado.

Se incluye una función `bytes2matrix` para convertir nuestro bloque de texto simple inicial en una matriz de estado. Escriba una función `matrix2bytes` para convertir esa matriz nuevamente en bytes y envíe el texto simple resultante como bandera.

Archivos del desafío:
- [matrix.py](https://cryptohack.org/static/challenges/matrix_e1b463dddbee6d17959618cf370ff1a5.py)

Recursos:
- [YouTube: Explicación del cifrado AES Rijndael como una animación Flash](https://www.youtube.com/watch?v=gP4PqVGudtg)

In [ ]:
def bytes2matrix(text):
  """ Converts a 16-byte array into a 4x4 matrix.  """
  return [list(text[i:i+4]) for i in range(0, len(text), 4)]

def matrix2bytes(matrix):
  """ Converts a 4x4 matrix into a 16-byte array.  """
  return bytes([i for row in matrix for i in row])

matrix = [
  [99, 114, 121, 112],
  [116, 111, 123, 105],
  [110, 109, 97, 116],
  [114, 105, 120, 125],
]

matrix2bytes(matrix)

b'crypto{inmatrix}'

## Round Keys

Por ahora, vamos a pasar por alto los detalles más finos de la fase **KeyExpansion**. El punto principal es que toma nuestra clave de 16 bytes y produce 11 matrices de 4x4 llamadas "claves redondas" derivadas de nuestra clave inicial. Estas claves redondas permiten que AES aproveche al máximo la clave única que proporcionamos.

La fase **inicial de adición de claves**, que es la siguiente, consta de un único paso *AddRoundKey*. El paso *AddRoundKey* es sencillo: realiza una operación XOR entre el estado actual y la clave de ronda actual.

*AddRoundKey* también se produce como el paso final de cada ronda. *AddRoundKey* es lo que hace que AES sea una "permutación con clave" en lugar de solo una permutación. Es la única parte de AES donde la clave se mezcla con el estado, pero es crucial para determinar la permutación que ocurre.

Como has visto en desafíos anteriores, XOR es una operación que se puede invertir fácilmente si conoces la clave, pero es difícil de deshacer si no la conoces. Ahora imagina intentar recuperar texto sin formato que se ha XOR con 11 claves diferentes y que se ha mezclado mucho entre cada operación XOR con una serie de cifras de sustitución y transposición. ¡Eso es más o menos lo que hace AES! Y veremos cuán efectiva es la mezcla en los próximos desafíos.

Completa la función `add_round_key`, luego usa la función `matrix2bytes` para obtener tu próxima bandera.

Archivos del desafío:
* [add_round_key.py](https://cryptohack.org/static/challenges/add_round_key_b67b9a529ae739156107a74b14adde98.py)



In [ ]:
state = [
    [206, 243, 61, 34],
    [171, 11, 93, 31],
    [16, 200, 91, 108],
    [150, 3, 194, 51],
]

round_key = [
    [173, 129, 68, 82],
    [223, 100, 38, 109],
    [32, 189, 53, 8],
    [253, 48, 187, 78],
]

def matrix2bytes(matrix):
    """ Converts a 4x4 matrix into a 16-byte array.  """
    return bytes([i for row in matrix for i in row])

def add_round_key(s, k):
    return matrix2bytes([[s[i][j] ^ k[i][j] for j in range(len(s[i]))] for i in range(len(s))])

add_round_key(state, round_key)